In [1]:
import nibabel as nib
import numpy as np
import ipywidgets as widgets

In [2]:
im = nib.load("reviewUpsampledSegments/C4HalfGroundTruth.nii.gz")
GroundTruth = np.int8(im.get_fdata())

In [3]:
from scipy.ndimage import binary_erosion
from scipy.ndimage import zoom

def nearest_resample(v, factor):
    upsampled = zoom(v, zoom=factor, order=0)
    return (upsampled > 0.5).astype(np.uint8)

def linear_resample(v, factor):
    upsampled = zoom(v.astype(float), zoom=factor, order=1)
    return (upsampled > 0.5).astype(np.uint8)
    
def morphological_downsample(v, factor):
    eroded = binary_erosion(v)
    return nearest_resample(eroded, factor)

In [4]:
print(GroundTruth.shape)
downsampled_volume_0p5mm =  morphological_downsample(GroundTruth, factor=0.2)
downsampled_volume_0p2mm =  morphological_downsample(GroundTruth, factor=0.5)

downsampled_volume_0p5mm = np.float32(downsampled_volume_0p5mm)
downsampled_volume_0p2mm = np.float32(downsampled_volume_0p2mm)
print(downsampled_volume_0p5mm.shape)
print(downsampled_volume_0p2mm.shape)


(454, 443, 239)
(91, 89, 48)
(227, 222, 120)


In [5]:
#NN method accuracy
upsampled_NN_dx5 = np.float32(nearest_resample(downsampled_volume_0p5mm, factor = 5))
upsampled_NN_dx2 = np.float32(nearest_resample(downsampled_volume_0p2mm, factor = 2))

print(upsampled_NN_dx5.shape)
print(upsampled_NN_dx2.shape)

(455, 445, 240)
(454, 444, 240)


In [6]:
#Linear method accuracy
upsampled_linear_dx5 = np.float32(linear_resample(downsampled_volume_0p5mm, factor = 5))
upsampled_linear_dx2 = np.float32(linear_resample(downsampled_volume_0p2mm, factor = 2))

print(upsampled_linear_dx5.shape)
print(upsampled_linear_dx2.shape)

(455, 445, 240)
(454, 444, 240)


In [7]:
from SegmentationUpsampler import UpsampleMultiLabels
upsampled_Mesh_dx5 = UpsampleMultiLabels.upsample(downsampled_volume_0p5mm, [0.2, 0.2, 0.2], spacing=[1, 1, 1], sigma = 0.56, iso = 0.43, fillGaps=False, NB=True)
upsampled_Mesh_dx5 = np.float32(upsampled_Mesh_dx5)

upsampled_Mesh_dx2 = UpsampleMultiLabels.upsample(downsampled_volume_0p2mm, [0.5, 0.5, 0.5], spacing=[1, 1, 1], sigma = 0.61, iso = 0.3, fillGaps=False, NB=True)
upsampled_Mesh_dx2 = np.float32(upsampled_Mesh_dx2)

print(upsampled_Mesh_dx5.shape)
print(upsampled_Mesh_dx2.shape)

Label 1.0: Smoothed with σ=0.56
Label 1.0: Mesh extracted with iso=0.430
Label 1.0: Smoothed with σ=0.61
Label 1.0: Mesh extracted with iso=0.300
(455, 445, 240)
(454, 444, 240)


In [8]:
import numpy as np
from scipy import ndimage as ndi

def surface_mask(mask: np.ndarray, connectivity: int = 1):
    struct = ndi.generate_binary_structure(mask.ndim, connectivity)
    eroded = ndi.binary_erosion(mask, structure=struct)  # treats nonzero as True
    return np.logical_xor(mask.astype(bool), eroded)     # boundary voxels

def assd(gt: np.ndarray, pred: np.ndarray, spacing=None):
    if spacing is None:
        spacing = (1.0,) * gt.ndim

    Sg = surface_mask(gt)
    Sp = surface_mask(pred)

    if not Sg.any() and not Sp.any():
        return 0.0
    if not Sg.any() or not Sp.any():
        return np.inf

    dt_to_pred_surface = ndi.distance_transform_edt(~Sp, sampling=spacing)
    dt_to_gt_surface   = ndi.distance_transform_edt(~Sg, sampling=spacing)

    d_g2p = dt_to_pred_surface[Sg]
    d_p2g = dt_to_gt_surface[Sp]

    all_distances = np.concatenate([d_g2p, d_p2g])
    assd_mean = all_distances.mean()
    assd_std  = all_distances.std()

    return assd_mean#, assd_std #0.5 * (d_g2p.mean() + d_p2g.mean())

def evaluate(V):
    errors = np.sum(abs(V-GroundTruth))
    
    jaccard_val = errors/np.sum(V + GroundTruth == 2)
    dice_val = 2*np.sum(V + GroundTruth == 2)/np.sum(V + GroundTruth)
    percent_val = errors/np.sum(GroundTruth)
    plus_val = np.sum((V-GroundTruth)==1)
    minus_val = np.sum((V-GroundTruth)==-1)
    ASSD = assd(V, GroundTruth) #ASSD is relatively time consuming and can be commented in repetitive use 
    
    print(f"Jaccard: {jaccard_val:.4f}")
    print(f"Dice: {dice_val:.4f}")
    print(f"ASSD: {ASSD:.4f}")
    print(f"Percent: {percent_val:.4f}")
    print(f"Plus: {plus_val}, Minus: {minus_val}")

In [9]:
# for reviewers and users
# this is a post shifting process applied to upsampled 
# volumes, this process is important to ensure volumes
# have minimal position shift against the GroundTruth.
'''
vol_mesh = meshgood
for i in range(3):
    for j in range(3):
        for k in range(3):
            shifted = 0
            print(i-1, j-1, k-1)
            shifted = np.roll(vol_mesh, i-1, axis=0)
            shifted = np.roll(shifted,  j-1, axis=1)
            shifted = np.roll(shifted,  k-1, axis=2)
            evaluate(shifted)
'''

'\nvol_mesh = meshgood\nfor i in range(3):\n    for j in range(3):\n        for k in range(3):\n            shifted = 0\n            print(i-1, j-1, k-1)\n            shifted = np.roll(vol_mesh, i-1, axis=0)\n            shifted = np.roll(shifted,  j-1, axis=1)\n            shifted = np.roll(shifted,  k-1, axis=2)\n            evaluate(shifted)\n'

In [10]:
'''
NNdx5
-1 -1 0
Jaccard: 0.1059
Dice: 0.9497
Percent: 0.0979
Plus: 119524, Minus: 404663

NNdx2
0 0 -1
Jaccard: 0.0682
Dice: 0.9670
Percent: 0.0640
Plus: 9116, Minus: 333598

Lineardx5
-1 -1 0
Jaccard: 0.0820
Dice: 0.9606
Percent: 0.0766
Plus: 56951, Minus: 353321

Lineardx2
0 0 -1
Jaccard: 0.0606
Dice: 0.9706
Percent: 0.0572
Plus: 6789, Minus: 299763

Meshdx5
1 1 1
Jaccard: 0.0830
Dice: 0.9602
Percent: 0.0772
Plus: 42072, Minus: 371583

Meshdx2
1 0 0
Jaccard: 0.0347
Dice: 0.9830
Percent: 0.0337
Plus: 27717, Minus: 152581

'''

'\nNNdx5\n-1 -1 0\nJaccard: 0.1059\nDice: 0.9497\nPercent: 0.0979\nPlus: 119524, Minus: 404663\n\nNNdx2\n0 0 -1\nJaccard: 0.0682\nDice: 0.9670\nPercent: 0.0640\nPlus: 9116, Minus: 333598\n\nLineardx5\n-1 -1 0\nJaccard: 0.0820\nDice: 0.9606\nPercent: 0.0766\nPlus: 56951, Minus: 353321\n\nLineardx2\n0 0 -1\nJaccard: 0.0606\nDice: 0.9706\nPercent: 0.0572\nPlus: 6789, Minus: 299763\n\nMeshdx5\n1 1 1\nJaccard: 0.0633\nDice: 0.9693\nPercent: 0.0616\nPlus: 192229, Minus: 137808\n\nMeshdx2\n1 0 0\nJaccard: 0.0347\nDice: 0.9830\nPercent: 0.0337\nPlus: 27717, Minus: 152581\n\n'

In [11]:
volmesh = upsampled_NN_dx5[0:454, 0:443, 0:239]
i, j, k = -1, -1, 0
shifted = np.roll(volmesh, i, axis=0)
shifted = np.roll(shifted,  j, axis=1)
shifted = np.roll(shifted,  k, axis=2)

evaluate(shifted)
final_img = nib.Nifti1Image(np.int8(shifted), im.affine)
nib.save(final_img, r"reviewUpsampledSegments\C4NNUpsampleddx5.nii.gz")

Jaccard: 0.1059
Dice: 0.9497
ASSD: 1.2671
Percent: 0.0979
Plus: 119524, Minus: 404663


In [12]:
volmesh = upsampled_NN_dx2[0:454, 0:443, 0:239]
i, j, k = 0, 0, -1
shifted = np.roll(volmesh, i, axis=0)
shifted = np.roll(shifted,  j, axis=1)
shifted = np.roll(shifted,  k, axis=2)

evaluate(shifted)
final_img = nib.Nifti1Image(np.int8(shifted), im.affine)
nib.save(final_img, r"reviewUpsampledSegments\C4NNUpsampleddx2.nii.gz")

Jaccard: 0.0682
Dice: 0.9670
ASSD: 0.9401
Percent: 0.0640
Plus: 9116, Minus: 333598


In [13]:
volmesh = upsampled_linear_dx5[0:454, 0:443, 0:239]
i, j, k = -1, -1, 0
shifted = np.roll(volmesh, i, axis=0)
shifted = np.roll(shifted,  j, axis=1)
shifted = np.roll(shifted,  k, axis=2)

evaluate(shifted)
final_img = nib.Nifti1Image(np.int8(shifted), im.affine)
nib.save(final_img, r"reviewUpsampledSegments\C4LinearUpsampleddx5.nii.gz")

Jaccard: 0.0820
Dice: 0.9606
ASSD: 1.1221
Percent: 0.0766
Plus: 56951, Minus: 353321


In [14]:
volmesh = upsampled_linear_dx2[0:454, 0:443, 0:239]
i, j, k = 0, 0, -1
shifted = np.roll(volmesh, i, axis=0)
shifted = np.roll(shifted, j, axis=1)
shifted = np.roll(shifted, k, axis=2)

evaluate(shifted)
final_img = nib.Nifti1Image(np.int8(shifted), im.affine)
nib.save(final_img, r"reviewUpsampledSegments\C4LinearUpsampleddx2.nii.gz")

Jaccard: 0.0606
Dice: 0.9706
ASSD: 0.8759
Percent: 0.0572
Plus: 6789, Minus: 299763


In [15]:
volmesh = upsampled_Mesh_dx5[0:454, 0:443, 0:239]
i, j, k = 1, 1, 1
shifted = np.roll(volmesh, i, axis=0)
shifted = np.roll(shifted, j, axis=1)
shifted = np.roll(shifted, k, axis=2)

evaluate(shifted)
final_img = nib.Nifti1Image(np.int8(shifted), im.affine)
nib.save(final_img, r"reviewUpsampledSegments\C4MeshUpsampleddx5.nii.gz")

Jaccard: 0.0830
Dice: 0.9602
ASSD: 1.1703
Percent: 0.0772
Plus: 42072, Minus: 371583


In [16]:
volmesh = upsampled_Mesh_dx2[0:454, 0:443, 0:239]
i, j, k = 1, 0, 0
shifted = np.roll(volmesh, i, axis=0)
shifted = np.roll(shifted, j, axis=1)
shifted = np.roll(shifted, k, axis=2)

evaluate(shifted)
final_img = nib.Nifti1Image(np.int8(shifted), im.affine)
nib.save(final_img, r"reviewUpsampledSegments\C4MeshUpsampleddx2.nii.gz")

Jaccard: 0.0347
Dice: 0.9830
ASSD: 0.5504
Percent: 0.0337
Plus: 27717, Minus: 152581
